In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request
import os
import urllib.parse

In [2]:
fakeBlogKeywordList = ["스토리앤","seoulouba","mateb.kr","revu","weble","ohmyblog","mrblog","tble",
    "dinnerqueen","%EA%B3%B5%EC%A0%95%EA%B1%B0%EB%9E%98%EC%9C%84%EC%9B%90%ED%9A%8C-%EB%AC%B8%EA%B5%AC",
    "%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9B%90%EC%A0%95%EB%8C%80",
    "banner_","%EC%9D%B4%EC%8A%88%EB%B8%94%EB%A1%9C%EA%B7%B8",
    "%EB%B0%B0%EB%84%88","http://echoblog.net/images/sponsor-banner.png",
                      "sponsor","banner","echoblog"]

In [14]:
#이미지 저장
def saveImage(div_option,index,img_url,nickName,postNumber):
    #블로그 유저 별로 폴더를 나눌건지에 대한 옵션
    #true -> path : image/{nickName}/{postNumber}
    #false -> path : image/myimg/
    if(div_option):
        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"        
        imgPath = f'image/{nickName}/{postNumber}/{index}.jpg'
        dirPath = f'image/{nickName}/{postNumber}'
    #폴더별로 분기하지 않을떄 path : image/myimg/
    else:
        list = os.listdir("image/myimg")
        last_index = len(list)
#         print("last_index : ",last_index)
        dirPath = 'image/myimg'
        imgPath = f'image/myimg/{last_index}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
                
    #파일 저장
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)

        for item in fakeBlogKeywordList:
            if(img_url_final.find(item) > 0):
                print('this is fake img')
                list = os.listdir("ad_img")
                last_index = len(list)
                imgPath = f"ad_img/{last_index}.jpg"
                break;
        urllib.request.urlretrieve(img_url_final, imgPath)
        return True
    except Exception as ex:
        print("img save error : " , ex)
        return False
            

In [4]:
def findBlogImgGethering(searchQuery, searchOption, page):
    url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
    html = requests.get(url)
    # 1차, blog URL 찾기
    soup = BeautifulSoup(html.text, 'html.parser')
    index = 0;

    url_list = [];
    for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
        child_item = li_item.find('a',{'class' : 'sh_blog_title'})
        title = child_item.attrs['title']
        href = child_item.attrs['href']

        #url list 저장
        url_list.append(href)

        print(f'title : {title} , href : {href}')
        print('----------------------------------')
        
    # url 파싱 및 예외처리
    for url_item in url_list:
        nickName = "";
        postNumber = "";

        if(url_item.find("blog.me") > 0):
            parsing = url_item;
            parsing = parsing.replace("https://","")
            nickName = parsing.split('.')[0]
            postNumber = parsing.split('/')[1]
            blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber

        else:
            blogUrl = url_item.replace("https://", "https://m.");

        #https://m.blog.naver.com/soeyoung1004?Redirect=Log&logNo=221944814459

        nickName = blogUrl.split('/')[-1].split('?')[0]
        postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]

        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"
            
        print("------ digging more -------")
        print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
        print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
        print("blogUrl : " , blogUrl)        

        #전체 목록 순회
        blog_html = requests.get(blogUrl)
        blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
        blog_image_class = blog_soup.find_all('div',{'class' : 'se-image'});

        for div_obj in blog_image_class:
            for idx, img_item in enumerate(div_obj.find_all('img')):
                img_url = img_item.attrs['src'];

                if(img_url.find("w80_blur") > 0):
                    img_url = img_url.replace("w80_blur" , "w800")

                isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
                if(isSaveSuccess):  
                    index = index+1
#                    print('index  : ' , index , "blogUrl : " + blogUrl)
#                 if not os.path.isdir(f'image/{nickName}/{postNumber}'):
#                     try:
#                         os.makedirs(f'image/{nickName}/{postNumber}')

#                     except Exception as ex:
#                         print("error catch : " , ex)
#                 try:
#                     filename = img_url.split('/')[-1].split('?')[0];
#                     img_url2 = urllib.parse.quote_plus(str(filename))
#                     img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
#                     urllib.request.urlretrieve(img_url_final, f'image/{nickName}/{postNumber}/{index}.jpg')
#                     index = index +1;
#                 except Exception as ex:
#                     print("img save error : " , ex)


In [ ]:
searchQuery = "샤로수길 맛집"
searchOption = 'sim'  #sim or date  
page = 4

findBlogImgGethering(searchQuery,searchOption,page)

title : 감탄했던 샤로수길 맛집 , href : https://blog.naver.com/blueblublue?Redirect=Log&logNo=221825601192
----------------------------------
title : 샤로수길맛집 소블리 서울대입구맛집 고기 전문점으로 최고 , href : https://blog.naver.com/poi6240?Redirect=Log&logNo=221970216847
----------------------------------
title : 또 가고싶은 샤로수길 맛집! , href : https://blog.naver.com/pocoholl?Redirect=Log&logNo=221959156002
----------------------------------
title : 샤로수길 맛집! 달달한 돼지갈비가 맛있는 고기굽는 사람들! , href : https://choicemmcs.blog.me/221997186705
----------------------------------
title : 인상깊었던 샤로수길 맛집 , href : https://blog.naver.com/blue2489?Redirect=Log&logNo=221952462488
----------------------------------
title : 두번 가고싶은 샤로수길 맛집 , href : https://blog.naver.com/ehowlnn0?Redirect=Log&logNo=221816583784
----------------------------------
title : 샤로수길 맛집 도리도리하찌 찐맛 닭볶음탕 , href : https://blog.naver.com/haedud128?Redirect=Log&logNo=221990175261
----------------------------------
title : 샤로수길 맛집 온돌에서 깔끔한 한끼 , href : https://qkqhtjwogus.blog.m